# Step 1: mechanics with Euler-Lagrange

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
init_vprinting()

In [ ]:
m0, m1, l0, l1, t, g = symbols('m_0 m_1 l_0 l_1 t g')

In [ ]:
the1, the2 = dynamicsymbols('theta_1 theta_2')
the1, the2

In [ ]:
thedot1 = the1.diff(t)
thedot2 = the2.diff(t)
thedot1, thedot2

In [ ]:
thedotdot1 = thedot1.diff(t)
thedotdot2 = thedot2.diff(t)
thedotdot1, thedotdot2

In [ ]:
T = m0*l0**2*thedot1**2/2 + m1*(l0**2*thedot1**2 + l1**2*thedot2**2 + 2*l0*l1*thedot1*thedot2*cos(the2-the1))/2
T

In [ ]:
U = -m0*g*cos(the1)*l0 - m1*g*(l0*cos(the1)+l1*cos(the2))
U

In [ ]:
L = T-U
L

In [ ]:
L.expand()

In [ ]:
E = T + U
E

In [ ]:
L.diff(thedot1).simplify().diff(t).simplify()

In [ ]:
L.diff(the1)

In [ ]:
def lagrange(L_expr, var):
    vardot = var.diff(t)
    lag1 = L_expr.diff(vardot).simplify().diff(t).simplify()
    lag2 = L_expr.diff(var).simplify()
    lag = lag1-lag2
    return lag.simplify().expand()

In [ ]:
def get_lagr_eqs(T, U, n_vars):
    '''Get a list of lagrange expressions. T and U are Kinetic energy
    and Potential energy, as functions of coordinates q, its
    derivatives and other parameters'''
    L = T-U
    res = []
    for ii in range(n_vars):
        q = dynamicsymbols(f'q_{ii}')
        res.append(lagrange(L, q))
    return res

In [ ]:
l_expr_1 = lagrange(L, the1)
l_expr_1

In [ ]:
l_expr_1.expand()

In [ ]:
wikiexpr = (
    (m0+m1)*l0**2*thedotdot1 +
    m1*thedotdot2*l0*l1*cos(the1-the2) +
    -m1*thedot2*l0*l1*(thedot1-thedot2)*sin(the1-the2) + 
    m1*thedot1*thedot2*l0*l1*sin(the1-the2) + 
    (m0+m1)*g*l0*sin(the1))
wikiexpr

In [ ]:
resto = wikiexpr-l_expr_1
resto.simplify()

## Transformando la eq de Lagrange en fórmula para control

In [ ]:
v1, v2, a1, a2 = symbols('v_1 v_2 a_1 a_2')
u1, u2 = symbols('u_1 u_2')

In [ ]:
l_expr_1_sims = l_expr_1.expand().subs((
    [thedotdot1, a1],
    [thedotdot2, a2],
    [thedot1, v1],
    [thedot2, v2]
))
l_expr_1_sims

In [ ]:
c1_a1 = l_expr_1_sims.collect(a1).coeff(a1)
c1_a1

In [ ]:
c1_a2 = l_expr_1_sims.collect(a2).coeff(a2)
c1_a2

In [ ]:
c1 = simplify(l_expr_1_sims- c1_a1 * a1 - c1_a2 * a2)
c1

In [ ]:
l_expr_2 = lagrange(L, the2)
l_expr_2_sims = l_expr_2.expand().subs((
    [thedotdot1, a1],
    [thedotdot2, a2],
    [thedot1, v1],
    [thedot2, v2]
))
l_expr_2_sims

In [ ]:
c2_a1 = l_expr_2_sims.collect(a1).coeff(a1)
c2_a1

In [ ]:
c2_a2 = l_expr_2_sims.collect(a2).coeff(a2)
c2_a2

In [ ]:
c2 = simplify(l_expr_2_sims- c2_a1 * a1 - c2_a2 * a2)
c2

In [ ]:
mat_acc = Matrix([
    [c1_a1, c1_a2],
    [c2_a1, c2_a2]
])
mat_acc

In [ ]:
acc_mat = Matrix([
    [a1],
    [a2]
])
c_mat = Matrix([
    [c1],
    [c2]
])
u_mat = Matrix([
    [u1],
    [u2]
])
acc_mat, c_mat, u_mat

In [ ]:
RHS = simplify(mat_acc.inv() @ (u_mat - c_mat))
RHS

In [ ]:
RHS.subs([
    [l0, 1],
    [l1, 1],
    [m0, 1],
    [m1, 1],
    [g, 1]
])

In [ ]:
RHS.shape[0]

In [ ]:
def diff_to_symb(expr, n_var):
    '''Transform an expression with derivatives to symbols'''
    t = symbols('t')
    for jj in range(n_var):
        q = dynamicsymbols(f'q_{jj}')
        expr = expr.subs((
            [q.diff(t,2), symbols(f'a_{jj}')],
            [q.diff(t), symbols(f'v_{jj}')]
        ))
    return expr

In [ ]:
l_expr_1_q = l_expr_1.subs(the1, dynamicsymbols('q_0')).subs(the2, dynamicsymbols('q_1'))
l_expr_2_q = l_expr_2.subs(the1, dynamicsymbols('q_0')).subs(the2, dynamicsymbols('q_1'))
T_q = T.subs(the1, dynamicsymbols('q_0')).subs(the2, dynamicsymbols('q_1'))
U_q = U.subs(the1, dynamicsymbols('q_0')).subs(the2, dynamicsymbols('q_1'))
RHS_q = RHS.subs([
    [the1, symbols('q_0')],
    [the2, symbols('q_1')],
    [symbols('u_1'), symbols('u_0')],
    [symbols('u_2'), symbols('u_1')],
    [symbols('v_1'), symbols('v_0')],
    [symbols('v_2'), symbols('v_1')],
])
    

In [ ]:
get_lagr_eqs(T_q, U_q, 2)

In [ ]:
diff_to_symb(l_expr_1_q,2)

In [ ]:
def lagr_to_RHS(lagr_eqs):
    '''Takes lagrangian equations,
    Calculates the Right Hand Side functions of
    the second order derivatives as used in optimal control'''
    n_var = len(lagr_eqs)
    coeff_mat = []
    acc_mat = []
    c_mat = []
    u_mat = []
    for ii in range(n_var):
        expr = diff_to_symb(lagr_eqs[ii], n_var)
        coeff_line = []
        rest = expr
        for jj in range(n_var):
            a = symbols(f'a_{jj}')
            coeff_line.append(expr.collect(a).coeff(a))
            rest = rest - a*expr.collect(a).coeff(a)
        coeff_mat.append(coeff_line)
        acc_mat.append([symbols(f'a_{ii}'),])
        u_mat.append([symbols(f'u_{ii}'),])
        c_mat.append([simplify(rest),])
    coeff_mat = Matrix(coeff_mat)
    acc_mat = Matrix(acc_mat)
    c_mat = Matrix(c_mat)
    u_mat = Matrix(u_mat)
    RHS = simplify(coeff_mat.inv() @ (u_mat - c_mat))
    new_RHS = []
    for expr in RHS:
        for jj in range(n_var):
            expr = expr.subs(dynamicsymbols(f'q_{jj}'),symbols(f'q_{jj}'))
        new_RHS.append(expr)
    return Matrix(new_RHS)

In [ ]:
RHS = lagr_to_RHS(get_lagr_eqs(T_q, U_q, 2))

In [ ]:
RHS

In [ ]:
RHS_q

In [ ]:
simplify(RHS - RHS_q)

In [ ]:
varset = RHS.atoms(Symbol)
varset

In [ ]:
varset.__str__()[1:-1]

In [ ]:
RHS.__str__()

In [ ]:
_ = np.ones([10,3])
_

In [ ]:
_x, _y, _z = unpack(_)

In [ ]:
def print_funcs(RHS, n_var):
    RHS = list(RHS)
    q_args = []
    v_args = []
    u_args = []
    params = []
    args = []
    funcs = []
    for jj in range(n_var):
        q = symbols(f"q_{jj}")
        q_args.append(q)
        v = symbols(f"v_{jj}")
        v_args.append(v)
        u = symbols(f"u_{jj}")
        u_args.append(u)
        args += [q, v, u]
    x_args = q_args + v_args
    for ii in range(len(RHS)):
        expr = RHS[ii]
        var_set = expr.atoms(Symbol)
        for symb in var_set:
            if not symb in args:
                params.append(symb)
        funcs.append(expr)
        
    msg = 'def F(x, u, params):\n'
    msg += f'    {x_args.__str__()[1:-1]} = unpack(x)\n'
    msg += f'    {u_args.__str__()[1:-1]} = unpack(u)\n'
    msg += f'    {params.__str__()[1:-1]} = params\n'
    msg += f'    result = [{v_args.__str__()[1:-1]},]\n'
    for expr in funcs:
        msg += '    result.append('+ expr.__str__() + ')\n'
    msg += '\n    return result\n'
        
    print(msg)
    return msg

In [ ]:
print_funcs(RHS, 2)

In [ ]:
def f_0(x, u, params):
    q_0, q_1, v_0, v_1 = x
    u_0, u_1 = u
    m_1, l_1, l_0, m_0, g = params
    result = (l_0*(l_1*m_1*(g*sin(q_1) - l_0*v_0**2*sin(q_0 - q_1)) - u_1)*cos(q_0 - q_1) + l_1*(-l_0*(g*m_0*sin(q_0) + g*m_1*sin(q_0) + l_1*m_1*v_1**2*sin(q_0 - q_1)) + u_0))/(l_0**2*l_1*(m_0 - m_1*cos(q_0 - q_1)**2 + m_1))
    return result

def f_1(x, u, params):
    q_0, q_1, v_0, v_1 = x
    u_0, u_1 = u
    m_1, l_1, l_0, m_0, g = params
    result = (-l_0*(m_0 + m_1)*(l_1*m_1*(g*sin(q_1) - l_0*v_0**2*sin(q_0 - q_1)) - u_1) + l_1*m_1*(l_0*(g*m_0*sin(q_0) + g*m_1*sin(q_0) + l_1*m_1*v_1**2*sin(q_0 - q_1)) - u_0)*cos(q_0 - q_1))/(l_0*l_1**2*m_1*(m_0 - m_1*cos(q_0 - q_1)**2 + m_1))
    return result



## Euler discretization:

$$x_{n+1} = x_n + dt x'$$
$$x' = \frac{x_{n+1}-x_n}{dt}$$

In [ ]:
thedot1_1 = dynamicsymbols('theta_1n+1').diff(t)
thedot2_1 = dynamicsymbols('theta_2n+1').diff(t)
thedot1_1, thedot2_1

In [ ]:
dt = symbols('dt')

In [ ]:
def lagrange_disc(expr, var, params):
    vardot = var.diff(t)
    vardotdot = vardot.diff(t)
    name = var.name
    lag1 = expr.diff(vardot).simplify().diff(t).simplify()
    lag2 = expr.diff(var).simplify()
    lag = (lag1-lag2).simplify()
    vardot_1 = dynamicsymbols(name+'n+1').diff(t)
    dt = symbols('dt')
    lag = lag.subs(vardotdot, (vardot_1-vardot)/dt)
    for para in params:
        parname = para.name
        pardot = para.diff(t)
        pardotdot = pardot.diff(t)
        pardot_1 = dynamicsymbols(parname+'n+1').diff(t)
        lag = lag.subs(pardotdot, (pardot_1-pardot)/dt)
    return lag.simplify()

In [ ]:
def discretize_dotdot(expr, params):
    for param in params:
        paradot = param.diff(t)
        paradotdot = paradot.diff(t)
        parname = param.name
        paradot_1 = dynamicsymbols(parname+'n+1').diff(t)
        expr = expr.subs(paradotdot, (paradot_1-paradot)/dt)
    return expr

In [ ]:
l_expr_1 = lagrange_disc(L, the1, [the2])
l_expr_1

In [ ]:
l_expr_1_alt = discretize_dotdot(lagrange(L, the1), [the1, the2])
l_expr_1_alt

In [ ]:
resto_2 = (l_expr_1-l_expr_1_alt)
resto.simplify()

In [ ]:
l_expr_2 = lagrange_disc(L, the2, [the1])
l_expr_2

# Step 2: conversion?

In [ ]:
import casadi as cas

In [ ]:
def sympy2casadi(sympy_expr,sympy_var,casadi_var):
    #import casadi
    assert casadi_var.is_vector()
    if casadi_var.shape[1]>1:
        casadi_var = casadi_var.T
    casadi_var = cas.vertsplit(casadi_var)
    from sympy.utilities.lambdify import lambdify

    mapping = {'ImmutableDenseMatrix': cas.blockcat,
             'MutableDenseMatrix': cas.blockcat,
             'Abs':cas.fabs
            }
    f = lambdify(sympy_var,sympy_expr,modules=[mapping, cas])
    print(casadi_var)
    return f(*casadi_var)

In [ ]:
l_sy_vars = [m0, m1, l0, l1, t, dt, g, the1, the2, thedot1, thedot2, thedot1_1, thedot2_1]

In [ ]:
x = cas.SX.sym('x', 4)
x_n = cas.SX.sym('x_n', 4)
u = cas.SX.sym('u', 2)
m = cas.SX.sym('m', 2)
l = cas.SX.sym('l', 2)
T = cas.SX.sym("t")
Dt = cas.SX.sym("dt")
G = cas.SX.sym("g")

In [ ]:
l_cas_vars = cas.vertcat(m[0],m[1],l[0],l[1],T,Dt,G,x[0],x[1],x[2],x[3],x_n[2], x_n[3])

In [ ]:
L_expr_1 = sympy2casadi(l_expr_1, l_sy_vars, l_cas_vars)
L_expr_1

In [ ]:
L_expr_2 = sympy2casadi(l_expr_2, l_sy_vars, l_cas_vars)
L_expr_2

In [ ]:
Energy = sympy2casadi(E, l_sy_vars, l_cas_vars)
Energy

In [ ]:
step_rest_1 = cas.Function('Step_rest_1', [x, x_n, u, Dt, m, l, G], [L_expr_1-u[0]])

In [ ]:
step_rest_2 = cas.Function('Step_rest_2', [x, x_n, u, Dt, m, l, G], [L_expr_2-u[1]])

In [ ]:
energ = cas.Function('Energy', [x, m, l, G], [Energy])

In [ ]:
euler_step = cas.Function('Euler_step', [x, Dt], [x[0]+Dt*x[2], x[1]+Dt*x[3]])

In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 200

In [ ]:
def create_anim(sol):
    fig, ax = plt.subplots()

    fig.set_size_inches([8,8])
    ax.set_xlim(( -2.5, 2.5))
    ax.set_ylim(( -2.5, 2.5))

    circle1 = plt.Circle((0, 0), 2, color='b', ls = ":", fill=False)
    circle2 = plt.Circle((0, 0), 1, color='b', ls = ":", fill=False)
    ax.add_artist(circle1)
    ax.add_artist(circle2)

    line1, = ax.plot([], [], lw=2)
    line2, = ax.plot([], [], lw=2)
    line3, = ax.plot([], [], 'k', lw=1, ls = ':')
    point1, = ax.plot([], [], marker='o', markersize=15, color="red")
    point2, = ax.plot([], [], marker='o', markersize=15, color="red")
    text = ax.text(0.2, 0, "", fontsize = 12)
    text_2 = ax.text(0.2, -0.15, "", fontsize = 12)
    
    trayectory = [[0,-2],]
    
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        line3.set_data([], [])
        point1.set_data([], [])
        point2.set_data([], [])
        text.set_text('')
        return (line1,line2,)
    def animate(i):
        x1 = [0, np.sin(sol.value(X)[i,0])]
        y1 = [0, -np.cos(sol.value(X)[i,0])]
        x2 = [x1[1], x1[1]+np.sin(sol.value(X)[i,1])]
        y2 = [y1[1], y1[1]-np.cos(sol.value(X)[i,1])]
        trayectory.append([x2[1], y2[1]])
        line1.set_data(x1, y1)    
        point1.set_data(x1[1], y1[1])
        line2.set_data(x2, y2)    
        point2.set_data(x2[1], y2[1])
        tray = np.array(trayectory)
        line3.set_data(tray[:,0], tray[:,1])    
        text.set_text("U = %.6f" % sol.value(U)[i,0])
        text_2.set_text(r"$\dot{\theta}$" + " = %.6f" % sol.value(X)[i,1])
        return (line1,line2,)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim

In [ ]:
N = 500

In [ ]:
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable(N+1,4)
U = opti.variable(N,2)
T = opti.parameter()
u_m = opti.parameter(2)

In [ ]:
M = opti.parameter(2)
L = opti.parameter(2)
G = opti.parameter()

In [ ]:
#cost = cas.sum1(2+cas.cos(X[:,0])+cas.cos(X[:,1]))*T
cost = cas.sum1(2+cas.cos(X[:,1]))*T
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0, 0, 0])
opti.subject_to(cas.cos(X[-1,0]) < -0.9999)
opti.subject_to(cas.cos(X[-1,1]) < -0.9999)
opti.subject_to(opti.bounded(-0.001,X[-1,2],0.001))
opti.subject_to(opti.bounded(-0.001,X[-1,3],0.001))

In [ ]:
for ii in range(N):
    _step = euler_step(X[ii,:], T/N)
    opti.subject_to(X[ii+1,0] == _step[0])
    opti.subject_to(X[ii+1,1] == _step[1])
    opti.subject_to(step_rest_1(X[ii,:], X[ii+1,:], U[ii,:], T/N, M, L, G) == 0)
    opti.subject_to(step_rest_2(X[ii,:], X[ii+1,:], U[ii,:], T/N, M, L, G) == 0)
    opti.subject_to(opti.bounded(-u_m[0],U[ii, 0],u_m[0]))
    opti.subject_to(opti.bounded(-u_m[1],U[ii, 1],u_m[1]))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, cas.pi, N+1))
opti.set_initial(X[:,2], np.linspace(0, cas.pi, N+1))
opti.set_initial(X[:,1], cas.pi/N)
opti.set_initial(X[:,3], cas.pi/N)
opti.set_value(T, 50)
max_par = 0.2
opti.set_value(u_m, [max_par, 0])
opti.set_value(L, [1,1])
opti.set_value(M, [1,1])
opti.set_value(G, [1])

In [ ]:
sol = opti.solve()

In [ ]:
xx = sol.value(X)
uu = sol.value(U)
ene = []
for ii in range(N+1):
    ene.append(energ(xx[ii,:], [1,1], [1,1], [1]))
ene = np.array(ene)
pot = uu[:,0] * xx[:-1,2] * 50/N
work = [-3]
for ii in pot:
    work.append(work[-1] + ii)
work = np.array(work)    

In [ ]:
plt.figure(figsize=[14,10])
plt.plot(xx[:,0], 'b')
plt.plot(xx[:,2], 'b:')
plt.plot(xx[:,1], 'orange')
plt.plot(xx[:,3], 'orange', ls = ':')
plt.plot(uu[:,0], 'g:')

In [ ]:
plt.figure(figsize=[14,10])
plt.plot(ene.flatten(), 'r:')
plt.plot(uu[:,0], 'g:')
plt.plot(uu[:,0] * xx[:-1,2], 'b:')
plt.plot(work, 'b-', linewidth = 1)
plt.hlines([0,np.pi, -np.pi], 0, N, 'k', 'dotted')

In [ ]:
anim = create_anim(sol)

In [ ]:
HTML(anim.to_jshtml())